# Using LaVague for QA Automation

In this notebook, we'll look at how LaVague can be used to generate automated tests from simple test scenario.

We will use LaVague to autonomously run the test and record xpath and actions. We'll then use an LLM to generate assert statements and the final reusable test file. 

We first define our test with the following variables

In [1]:
URL = "https://www.amazon.fr/"

GHERKIN = """Feature: Add and remove a single product from cart

  Scenario: Add a product to cart
    Given I am on the homepage
    When I click "Accepter" to accept cookies
    And I enter "Harry Potter et la Chambre des Secrets" into the search bar and press Enter
    And I click on "Harry Potter et la Chambre des Secrets" in the search results
    And I am on the product details page
    And I click on the "Ajouter au panier" button
    And I am taken to a confirmation page
    And I click on "Aller au panier"
    And I am taken to the cart page
    And I remove the product from the cart by clicking on "Supprimer"
    Then the cart should be empty
"""

In [2]:
FEATURE_FILE_NAME = "demo.feature" # name of the existing feature file
PYTEST_FILE_NAME = "demo_test.py" # name of the pytest file that will be generated

# We could make this framework agnostic by rewriting the prompt and examples passed to the multi-modal LLM
# TARGET_FRAMEWORK = "pytest-bdd"

### Parse a Gherkin and extract the last step of each scenarios

In [3]:
scenarios = GHERKIN.split("Scenario:")
parsed_scenarios = []
for scenario in scenarios[1:]:
    scenario_name, *scenario_steps = scenario.strip().split("\n")
    parsed_scenarios.append(
        {
            "name": scenario_name.strip(),
            "steps": [step.strip() for step in scenario_steps],
        }
    )

parsed_scenarios[0]

def prepare_scenario(scenario):
    return "\n".join(scenario["steps"]), scenario["steps"][-1]


steps, assert_statement = prepare_scenario(parsed_scenarios[0])


In [4]:
assert_statement

'Then the cart should be empty'

## Init LaVague

In [5]:
import base64
from io import BytesIO

from lavague.core import  WorldModel, ActionEngine
from lavague.core.agents import WebAgent
from lavague.drivers.selenium import SeleniumDriver
from lavague.core.retrievers import SemanticRetriever
from selenium.webdriver.chrome.options import Options


from llama_index.llms.openai import OpenAI
from llama_index.multi_modal_llms.openai import OpenAIMultiModal
from llama_index.legacy.readers.file.base import SimpleDirectoryReader

In [6]:
# chrome_options = Options()
# chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

# selenium_driver = SeleniumDriver(headless=False, options = chrome_options)
selenium_driver = SeleniumDriver(headless=False)
world_model = WorldModel()
action_engine = ActionEngine(selenium_driver)
agent = WebAgent(world_model, action_engine)

2024-07-26 15:01:44,735 - INFO - Screenshot folder cleared


Load the URL

In [7]:
agent.get(URL)

## Run the agent
Run the high level test case once. Get logs as a dataframe. 

In [8]:
agent.run(f"Run this test case: {steps}", log_to_db=True)

logs = agent.logger.return_pandas()

2024-07-26 15:02:00,846 - INFO - Thoughts:
- The current screenshot shows the homepage of Amazon.fr with a cookie consent banner.
- The first step in the test case is to accept cookies by clicking on the "Accepter" button.
- The "Accepter" button is visible and should be clicked to proceed with the test case.

Next engine: Navigation Engine
Instruction: Click on the "Accepter" button to accept cookies.
2024-07-26 15:02:23,523 - INFO - ### Thoughts:
- The current screenshot shows the homepage of Amazon.fr.
- The objective is to perform a series of actions to test the website's functionality, starting with entering "Harry Potter et la Chambre des Secrets" into the search bar and pressing Enter.
- The search bar is visible in the screenshot, and the next step is to interact with it.

### Next Engine:
Navigation Engine

### Instruction:
Enter "Harry Potter et la Chambre des Secrets" into the search bar and press Enter.
2024-07-26 15:02:54,598 - INFO - Thoughts:
- The current screenshot sho

Connected to SQLite
Table created or altered successfully
Log insert complete


## Extract navigation data from the agent run
### Get relevant nodes to generate the assert statement

In [ ]:
retriever = SemanticRetriever(xpathed_only=True)
html = selenium_driver.get_html()

nodes = retriever.retrieve(f"Verify this condition: {assert_statement}" , html.splitlines())

nodes

In [ ]:
from IPython.display import HTML

# display(HTML(nodes[0]))
for e in nodes: 
    display(HTML(e))

    

### Get all actions performed and screenshot of last page state

From logs, clean the "code" column to remove CoT comments then create a new dataframe that contains each instruction with the associated xpath and action taken

In [11]:
# get all actions and cleaup
actions = "\n".join(logs["code"].dropna())

def remove_comments(code):
    return '\n'.join([line for line in code.split('\n') if not line.strip().startswith('#')])

logs['action'] = logs['code'].dropna().apply(remove_comments)
cleaned_logs = logs[['instruction', 'action']].fillna('')
actions = '\n\n'.join(cleaned_logs['instruction'] + ' ' + cleaned_logs['action'])

instructions = "\n".join(cleaned_logs['instruction'])
print(actions)

# get last page screenshot
last_page_screenshot = SimpleDirectoryReader(logs.iloc[-1]["screenshots_path"]).load_data() # load last screenshot taken


Click on the "Accepter" button to accept cookies. 
- actions:
    - action:
        args:
            xpath: "/html/body/div/span/form/div[2]/span/span/input"
        name: "click"

Enter "Harry Potter et la Chambre des Secrets" into the search bar and press Enter. 
- actions:
    - action:
        args:
            xpath: "/html/body/div/header/div/div/div[2]/div/form/div[3]/div/input"
            value: "Harry Potter et la Chambre des Secrets"
        name: "setValue"
    - action:
        args:
            xpath: "/html/body/div/header/div/div/div[2]/div/form/div[3]/div/input"
            value: "Harry Potter et la Chambre des Secrets"
        name: "setValueAndEnter"

Click on "Harry Potter et la Chambre des Secrets" in the search results. 
- actions:
    - action:
        args:
            xpath: "/html/body/div/div/div/div/div/span/div/div[3]/div/div/span/div/div/div/span/a"
            value: ""
        name: "click"

Click on the "Ajouter au panier" button. 
- actions:
    - ac

# Output generation
Use all gathered data about the site to generate a Gherkin, then a pytest-bdd file

We'll first create two LLMs
- multi modal LLM `gpt-4o` is used to generate the final code file (we pass a screenshot of the last page state to help with the assert generation)
- text only LLM `gpt-4` to generate the feature file

In [12]:
gpt4o = OpenAIMultiModal("gpt-4o")
gpt4o.max_new_tokens = 2000 # 300 by default, we increase it to make sure our pytest file doesn't get trucated

gpt4 = OpenAI("gpt-4")

# we'll clean the triple quote answer from the LLMs
def clean_output(markdown_code_block):
    return markdown_code_block.replace("```python", "").replace("```", "").replace("```\n", "")

### Generate pytest file

In [ ]:
SYSTEM_PROMPT = f"""
You are an expert in software testing frameworks and Python code generation. You answer in python markdown only and nothing else, don't include anything after the last backticks. Your task is to:
1. Process a test case, existing actions (with their associated xpath) that was ran during the test by an agent, relevant HTML nodes, and the screenshot of the last state of the page.
2. Generate an assert statement for the last condition of the test case. THis is the most important, you need to generate one assert statement based on the provided `Then` step. 
3. Package everything in a pytest-bdd file following best practices. 
Requirements:
- Use descriptive function names and name the scenario appropriately.
- Use execute_script to handle potential ElementClickInterceptedException during clicks.
- Include fixtures, scenario, Gherkin-style definitions, etc.
- Use try-except blocks to catch exceptions and raise pytest.fail for assert condition steps as needed.
- Do not generate asserts for 'Given', 'When', or 'And' steps; only generate asserts for 'Then' steps.
- Always use provided selenium code that was already executed to find valid selectors for the final pytestfile. 
- You answer in python code only and nothing else.
"""

EXAMPLES = """
import pytest
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pytest_bdd import scenarios, given, when, then
# Constants
BASE_URL = 'https://form.jotform.com/241472287797370'
# Scenarios
scenarios('test_form_submission.feature')
# Fixtures
@pytest.fixture
def browser():
    driver = webdriver.Chrome()
    driver.implicitly_wait(10)
    driver.get(BASE_URL)
    yield driver
    driver.quit()
# Steps
@given('I am on the job application page')
def i_am_on_the_job_application_page(browser):
    pass
@when('I enter "John" in the "First Name" field')
def i_enter_first_name(browser):
    first_name_field = browser.find_element(By.XPATH, "/html/body/form/div[1]/ul/li[2]/div/div/span[1]/input")
    first_name_field.send_keys("John")
@when('I enter "Doe" in the "Last Name" field')
def i_enter_last_name(browser):
    last_name_field = browser.find_element(By.XPATH, "/html/body/form/div[1]/ul/li[2]/div/div/span[2]/input")
    last_name_field.send_keys("Doe")
@when('I enter "john.doe@example.com" in the "Email Address" field')
def i_enter_email_address(browser):
    email_field = browser.find_element(By.XPATH, "/html/body/form/div[1]/ul/li[3]/div/span/input")
    email_field.send_keys("john.doe@example.com")
@when('I enter "(123) 456-7890" in the "Phone Number" field')
def i_enter_phone_number(browser):
    phone_number_field = browser.find_element(By.XPATH, "/html/body/form/div[1]/ul/li[4]/div/span/input")
    phone_number_field.send_keys("(123) 456-7890")
@when('I leave the "Cover Letter" field empty')
def i_leave_cover_letter_empty():
    # No action needed as the field should remain empty
    pass
@when('I click the "Apply" button')
def i_click_apply_button(browser):
    apply_button = WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.XPATH, "/html/body/form/div[1]/ul/li[6]/div/div/button"))
    )
    browser.execute_script("arguments[0].scrollIntoView(true);", apply_button)
    apply_button.click()
@then('I should see an error message for the "Cover Letter" field')
def i_should_see_error_message(browser):
    try:
        error_message = browser.find_element(By.XPATH, "/html/body/form/div[1]/ul/li[5]/div/div/span")
        assert error_message.is_displayed()
    except Exception as e:
        pytest.fail(f"Error message not displayed: {e}")
"""



PROMPT = f"""{SYSTEM_PROMPT}

Generate a valid pytest-bdd file with the following inputs and examples to guide you:
Feature file name: {FEATURE_FILE_NAME}\n
URL: {URL}\n
Full feature to be tested: {GHERKIN}\n
Already executed (instruction + actions):\n{actions}\n
Assert statement (Then) to test: {assert_statement}
Chunk of HTML that may be usefull to generate the assert statement. Generate a general statement that checks for the assert using these nodes: {nodes}\n
Examples:\n\n{EXAMPLES}
"""

print(PROMPT)

### Generate the automated test

In [15]:
generated_pytest = gpt4o.complete(PROMPT, image_documents=last_page_screenshot).text
generated_pytest = clean_output(generated_pytest)
print(generated_pytest)


import pytest
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from pytest_bdd import scenarios, given, when, then

# Constants
BASE_URL = 'https://www.amazon.fr/'

# Scenarios
scenarios('demo.feature')

# Fixtures
@pytest.fixture
def browser():
    driver = webdriver.Chrome()
    driver.implicitly_wait(10)
    driver.get(BASE_URL)
    yield driver
    driver.quit()

# Steps
@given('I am on the homepage')
def i_am_on_the_homepage(browser):
    pass

@when('I click "Accepter" to accept cookies')
def i_click_accept_cookies(browser):
    try:
        accept_button = WebDriverWait(browser, 10).until(
            EC.element_to_be_clickable((By.XPATH, "/html/body/div/span/form/div[2]/span/span/input"))
        )
        browser.execute_script("arguments[0].scrollIntoView(true);", accept_button)
        accept_button.click()
    except Exception a

### Cleanup and write test to file

We'll cleanup and write two generated files to disk: 
- `.feature` contains the test scenarios written in the Gherkin syntax
- `.py` the actual automated test that we'll run with pytest

In [16]:
import os
with open(FEATURE_FILE_NAME, "w") as file:
        file.write(GHERKIN)
        
with open(PYTEST_FILE_NAME, "w") as file:
        print("WRITING FILE")
        file.write(generated_pytest)

WRITING FILE


## Run tests


In [17]:
!pytest demo_test.py

============================= test session starts ==============================
platform darwin -- Python 3.10.14, pytest-8.2.1, pluggy-1.5.0
rootdir: /Users/palmi/work/repos/rework-qa-automation/LaVague
configfile: pyproject.toml
plugins: anyio-4.3.0, bdd-7.1.2
collected 1 item                                                               

demo_test.py .                                                           [100%]

============================== 1 passed in 33.72s ==============================


Notes: 
- When loading the URL, I sometimes get a mobile version, sometimes a responsive desktop version. It can go both ways: 1st load for agent is the desktop, 2nd run with pytest is the mobile version: this breaks the test. No idea why. Can't reproduce. 
- gener